## **UAOPiloto**
Detección de estudiantes que van a desertar en 2 períodos.

TABLAS BASE:  DATOS

TABLAS MYSQL:
1. estudiantes_limpio:  es la tabla ESTUDIANTE pero ahora si limpia
2. periodosRegulares:   es la tabla de periodos y ya viene con un orden que me permite identificar " los siguientes dos periodos"
3.  

### **1.  Conexión a la base de datos**
Nos conectamos a la base de datos MYSQL

In [40]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)

### **2.  crear DATOS

Se crea DATOS a partir de DISTINCT EstudianteAsignatura(id_estudiante,periodo)

In [41]:
query = """
    SELECT distinct id_estudiante, periodo
    FROM estudianteAsignatura
    ORDER BY id_estudiante, periodo
"""
DATOS = pd.read_sql(query, con=motor)

# NOTA:  falta manejar el caso de cursos de verano...  cuándo marcar al estudainte como desertor?  
# ERIK ya tiene la pregunta 20231009. R/  indica ERIK solo considerar períodos regulares

### **2.1 adicionamos el primer periodo que hizo matricula**
De la tabla EstudiantesMatriculas adicionamos el primer periodo en que el estudiante hizo su priemra matricula
EstudiantesMatriculas(id_estudiante, MIN(periodo))

In [42]:
# adicionamos el primer periodo en que ese estudiante hizo una matricula
query = """
    SELECT  id_estudiante, min(periodo) periodo_inicial
    FROM EstudiantesMatriculas
    WHERE periodo like '%1' or periodo like '%3'
    GROUP BY id_estudiante
"""
temp = pd.read_sql(query, con=motor)
# hacemos join entre DATOS y PRIMER_PERIODO para adicionar el periodo_inicial
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.2 Adicionamos fecha_grado, fecha_nacimiento, genero, colegio, ciudad_colegio**
De la tabla Estudiantes_Limpia saco esta información por estudiante y la adicionamos a DATOS

In [43]:
# ahora adicionamos el periodo de grado Y TODAS LAS COLUMNAS REQUERIDAS DESDE ESTUDIANTES_LIMPIO 
# pues ya hice todo el trabajo
# de limpieza.

query = """
    SELECT  id_estudiante, fecha_grado , fecha_nacimiento, genero, colegio, ciudad_colegio
    FROM estudiantes_limpio
"""
temp = pd.read_sql(query, con=motor)

# hacemos join con DATOS para adicionar la fecha de grado
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.3 adicionamos el número del periodo (periodo_orden)**

en la tabla PERIODOS está el orden de cada periodo.  vamos a adicionarlos.


In [62]:
query = """
    SELECT  pr.periodo, pr.orden periodo_orden, pr.fecha_inicio periodo_fi, pr.fecha_final periodo_ff
    from periodosRegulares pr
"""
temp = pd.read_sql(query, con=motor)

DATOS = DATOS.merge(temp, on=['periodo'], how='left')


In [63]:
DATOS.columns

Index(['id_estudiante', 'periodo', 'periodo_inicial', 'fecha_grado',
       'fecha_nacimiento', 'genero', 'colegio', 'ciudad_colegio',
       'periodo_orden', 'periodo_fi', 'periodo_ff'],
      dtype='object')

In [64]:
DATOS['fecha_grado']

0            None
1      2018-12-01
2            None
3      2019-04-06
4            None
          ...    
827          None
828          None
829          None
830          None
831          None
Name: fecha_grado, Length: 832, dtype: object

SALVEMOS DATOS para que no nos toque procesar lo anterior

In [65]:
DATOS.to_excel('DATOS.xlsx')

### **2.3 Ahora si calculamos si dada una fila en d de DATOS, los siguientes 2 periodos no matriculó (ni se graduó)**

Ahora procedemos a generar LABEL: 1- el estudiante desertó.  0- el estudiante no desertó

hacemos un FOR en DATOS
por cada d (en el periodo i)
   miramos si i e i+1 tiene matricula: 
   si si:   lo marcamos como 0 -  
   si no:  miramos si se graduó en i, i+1 o i+2.  
           si si:  marcamos como 0
           si no:  marcamos como 1
   

In [67]:
DATOS = pd.read_excel('DATOS.xlsx')

In [99]:
# el máximo periodo_orden es 12

from datetime import datetime, timedelta

DATOS['LABEL'] = -1   # asumimos que no se puede calcular

for indice, d in DATOS.iterrows():
    
    id_estudiante = d['id_estudiante']
    periodo_orden = d['periodo_orden']

    # print(id_estudiante, periodo_orden,'fecha_grado:', d['fecha_grado'], 'periodo_fi:', d['periodo_fi'], d['periodo_fi'] + timedelta(days=365))


    if periodo_orden >= 10:  # si es 11 o 12 no podemos aún calcular si si o si no
        d['LABEL'] = -1
        # print('     no se sabe')
    else:
        existe_periodo_1 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +1)]
        existe_periodo_2 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +2)]

        # print('sem1:', len(existe_periodo_1), 'sem2:', len(existe_periodo_2))
        if len(existe_periodo_1)>0 or len(existe_periodo_2)>0:
            
            # print('     NO desertor')
            d['LABEL'] = 0  # si matriculó alguno de los dos periodos siguientes, NO es desertor.

        else: 

            if pd.isna(d['fecha_grado']):

                d['LABEL'] = 1  # si no tiene fecha de grado -> desertor
                # print('     DESERTOR')
            else:  
                
                if d['fecha_grado'] > d['periodo_fi'] + timedelta(days=365):  
                    d['LABEL'] = 1  # si tiene fecha de grado y es mayor a dos semestres es desertor
                    # print('  DESERTOR')
                else:
                    d['LABEL'] = 0   # auque no matriculo 2 siguientes semestres, se graduó antes del año. NO es desertor
                    # print('     NO desertor')


In [100]:
DATOS.columns

Index(['Unnamed: 0', 'id_estudiante', 'periodo', 'periodo_inicial',
       'fecha_grado', 'fecha_nacimiento', 'genero', 'colegio',
       'ciudad_colegio', 'periodo_orden', 'periodo_fi', 'periodo_ff', 'LABEL'],
      dtype='object')

In [102]:
DATOS['LABEL'].value_counts()

LABEL
-1    832
Name: count, dtype: int64